# Working details of SSD

Jusqu'à présent, nous avons vu un scénario dans lequel nous avons fait des prédictions après avoir progressivement convolé et fait de pooling à la sortie de la couche précédente. Cependant, nous savons que différentes couches ont des champs récepteurs différents de l'image d'origine. Par exemple, les couches initiales ont un champ récepteur plus petit par rapport aux couches finales, qui ont un champ récepteur plus grand. Ici, nous allons apprendre comment le SSD tire parti de ce phénomène pour proposer une prédiction de cadres de délimitation pour les images.

Le fonctionnement derrière la façon dont SSD aide à surmonter le problème de la détection d'objets à différentes échelles est le suivant:
* 
Nous exploitons le réseau VGG pré-entraîné et l'étendons avec quelques couches supplémentaires jusqu'à ce que nous obtenions un bloc 1 x 1

* Au lieu d'exploiter uniquement la couche finale pour les prédictions de boîte englobante et de classe, nous tirerons parti de toutes les dernières couches pour faire des prédictions de classe et de boîte englobante.

* À la place des boîtes d'ancrage, nous proposerons des boîtes par défaut qui ont un ensemble spécifique d'échelle et de proportions.


* Chacune des boîtes par défaut doit prédire le décalage de l'objet et du cadre de délimitation, tout comme la façon dont les boîtes d'ancrage sont censées prédire les classes et les décalages dans YOLO

Maintenant que nous comprenons les principales différences entre SSD et YOLO (c'est-à-dire que les boîtes par défaut dans SSD remplacent les boîtes d'ancrage dans YOLO et que plusieurs couches sont connectées à la couche finale dans SSD, au lieu d'un pool de convolution progressif dans YOLO), apprenons ce qui suit:

* L'architecture réseau du SSD
* Comment tirer parti de différentes couches pour les prédictions de cadre de délimitation et de classe
* Comment attribuer une échelle et des proportions pour les boîtes par défaut dans différentes couches.



<img src='https://media.springernature.com/original/springer-static/image/chp%3A10.1007%2F978-981-15-3270-2_33/MediaObjects/482669_1_En_33_Fig1_HTML.png' width=700px>




Comme vous pouvez le voir dans le schéma précédent, nous prenons une image de taille 300 x 300 x 3 et la faisons passer à travers un réseau VGG-16 pré-entraîné pour obtenir la sortie de la couche conv5_3. De plus, nous étendons le réseau en ajoutant quelques convolutions supplémentaires à la sortie conv5_3.

Ensuite, nous obtenons un décalage de boîte englobante et une prédiction de classe pour chaque cellule et chaque boîte par défaut (plus sur les boîtes par défaut dans la section suivante ; pour l'instant, imaginons que cela soit similaire à une boîte d'ancrage). Le nombre total de prédictions provenant de la sortie conv5_3 est de 38 x 38 x 4, où 38 x 38 est la forme de sortie de la couche conv5_3 et 4 est le nombre de boîtes par défaut fonctionnant sur la couche conv5_3.

Voyons maintenant les différentes échelles et proportions des boîtes par défaut. Nous allons commencer par les échelles, puis passer aux proportions.

Imaginons un scénario où l'échelle minimale d'un objet est de 20 % de la hauteur et 20 % de la largeur d'une image, et l'échelle maximale de l'objet est de 90 % de la hauteur et 90 % de la largeur. Dans un tel scénario, nous augmentons progressivement l'échelle entre les couches (à mesure que nous progressons vers les couches ultérieures, la taille de l'image diminue considérablement), comme suit:
<img src='https://lilianweng.github.io/lil-log/assets/images/SSD-box-scales.png' width=700px>


La formule qui permet la mise à l'échelle progressive de l'image est la suivante:

$$lavel \ index:=1,...,L$$

$$scale\ of\ boxes: s_l=s_{min}+\frac{s_{max}-s_{smin}}{L-1}(l-1)$$

Maintenant que nous comprenons comment calculer l'échelle entre les couches, nous allons maintenant apprendre à créer des boîtes de différents rapports d'aspect.

Les rapports d'aspect possibles sont les suivants

$$aspect\ ratio: r\in \{1,2,3,0.5,0.33\}$$

Le centre de la boîte pour les différentes couches est le suivant :

$$ center\ location :(x_l^i, y_l^i)=(\frac{i+0.5}{m},\frac{j+0.5}{n}$$ 

Ici i et j représentent ensemble une cellule de la couche l

La largeur et la hauteur correspondant aux différents rapports d'aspect sont calculées comme suit:

$$widht: w_l^r =s_l\sqrt{r}$$
$$height: h_l^r =s_l\sqrt{r}$$

Notez que nous considérions quatre boîtes dans certaines couches et six boîtes dans une autre couche. Maintenant, si nous voulons avoir quatre cases, nous supprimons les proportions {3,1/3}, sinon nous considérons toutes les six cases possibles (cinq cases avec la même échelle et une case avec une échelle différente). Alors, apprenons comment nous obtenons la sixième case

$$additional\ scal: s_l^{'}\sqrt{s_ls_{l+1}}\ when\ r=1$$

Maintenant que nous avons toutes les cases possibles, comprenons comment nous préparons l'ensemble de données d'entraînement. Les cases par défaut qui ont une IoU supérieure à un seuil (disons, 0,5) sont considérées comme des correspondances positives, et les autres sont des correspondances négatives. Dans la sortie du SSD, nous prédisons la probabilité que la boîte appartienne à une classe (où la 0ème classe représente le fond) ainsi que le décalage de la vérité  par rapport à la boîte par défaut. Enfin, nous entraînons le modèle en optimisant les valeurs de pertes 

## Components in SSD code


Les fonctions utilitaires principales de cette section sont présentes dans le référentiel GitHub : https://github.com/sizhky/ssd-utils/  . Apprenons-les un par un avant de commencer le processus de formation

Il y a trois fichiers dans le référentiel GitHub. Examinons-les un peu et comprenons-les avant de nous entraîner. Notez que cette section ne fait pas partie du processus de formation, mais sert plutôt à comprendre les importations utilisées pendant la formation. Nous importons les classes SSD300 et MultiBoxLoss depuis le fichier model.py dans le référentiel GitHub. Apprenons-en tous les deux

### SSD300

In [ ]:
class SSD300(nn.Module):
    """
    The SSD300 network - encapsulates the base VGG network, auxiliary, and prediction convolutions.
    """

    def __init__(self, n_classes, device):
        super(SSD300, self).__init__()

        self.n_classes = n_classes
        self.device = device
        self.base = VGGBase()
        self.aux_convs = AuxiliaryConvolutions()
        self.pred_convs = PredictionConvolutions(n_classes)

        # Since lower level features (conv4_3_feats) have considerably larger scales, we take the L2 norm and rescale
        # Rescale factor is initially set at 20, but is learned for each channel during back-prop
        self.rescale_factors = nn.Parameter(torch.FloatTensor(1, 512, 1, 1))  # there are 512 channels in conv4_3_feats
        nn.init.constant_(self.rescale_factors, 20)

        # Prior boxes
        self.priors_cxcy = self.create_prior_boxes()
        self.to(device)

    def forward(self, image):
        """
        Forward propagation.
        :param image: images, a tensor of dimensions (N, 3, 300, 300)
        :return: 8732 locations and class scores (i.e. w.r.t each prior box) for each image
        """
        # Run VGG base network convolutions (lower level feature map generators)
        conv4_3_feats, conv7_feats = self.base(image)  # (N, 512, 38, 38), (N, 1024, 19, 19)

        # Rescale conv4_3 after L2 norm
        norm = conv4_3_feats.pow(2).sum(dim=1, keepdim=True).sqrt()  # (N, 1, 38, 38)
        conv4_3_feats = conv4_3_feats / norm  # (N, 512, 38, 38)
        conv4_3_feats = conv4_3_feats * self.rescale_factors  # (N, 512, 38, 38)
        # (PyTorch autobroadcasts singleton dimensions during arithmetic)

        # Run auxiliary convolutions (higher level feature map generators)
        conv8_2_feats, conv9_2_feats, conv10_2_feats, conv11_2_feats = \
            self.aux_convs(conv7_feats)  # (N, 512, 10, 10),  (N, 256, 5, 5), (N, 256, 3, 3), (N, 256, 1, 1)

        # Run prediction convolutions (predict offsets w.r.t prior-boxes and classes in each resulting localization box)
        locs, classes_scores = self.pred_convs(conv4_3_feats, conv7_feats, conv8_2_feats, conv9_2_feats, conv10_2_feats,
                                               conv11_2_feats)  # (N, 8732, 4), (N, 8732, n_classes)

        return locs, classes_scores

Nous envoyons d'abord l'entrée à VGGBase, qui renvoie deux vecteurs de caractéristiques de dimensions (N, 512, 38, 38) et (N, 1024, 19, 19). La deuxième sortie sera l'entrée pour AuxiliaryConvolutions, qui renvoie plus de cartes de caractéristiques de dimensions (N, 512, 10, 10), (N, 256, 5, 5), (N, 256, 3, 3) et (N , 256, 1, 1). Enfin, la première sortie de VGGBase et ces quatre featuremaps sont envoyées à PredictionConvolutions, qui renvoie 8 732 boîtes d'ancrage comme nous en avons discuté précédemment.

L'autre aspect clé de la classe SSD300 est la méthode create_prior_boxes. Pour chaque carte de caractéristiques, trois éléments lui sont associés : la taille de la grille, l'échelle de réduction de la cellule de la grille (il s'agit de la boîte d'ancrage de base pour cette carte de caractéristiques) et les proportions pour toutes les ancres d'une cellule. À l'aide de ces trois configurations, le code utilise une triple boucle for et crée une liste de (cx, cy, w, h) pour les 8 732 ancres.

Enfin, la méthode detect_objects prend des tenseurs de classification et des valeurs de régression (des boîtes d'ancrage prédites) et les convertit en coordonnées réelles de la boîte englobante

# MultiBoxLoss

En tant qu'humains, nous ne sommes préoccupés que par une poignée de cadres de délimitation. Mais pour le fonctionnement de SSD, nous devons comparer 8 732 cadres de délimitation de plusieurs cartes de caractéristiques et prédire si un cadre d'ancrage contient ou non des informations précieuses.

 Nous assignons cette tâche de calcul de perte à MultiBoxLoss. L'entrée pour la méthode directe est les prédictions de la boîte d'ancrage du modèle et les boîtes englobantes de la vérité. la boîte englobante.
 
Si l'IoU est suffisamment élevé, cette boîte d'ancrage particulière aura des coordonnées de régression non nulles et associera un objet comme vérité terrain pour la classification. Naturellement, la plupart des boîtes d'ancrage calculées auront leur classe associée en arrière-plan car leur IoU avec la boîte englobante réelle sera minuscule ou, dans de nombreux cas, nulle.

Une fois les vérités converties en ces 8 732 tenseurs de régression et de classification des boîtes d'ancrage, il est facile de les comparer aux prédictions du modèle puisque les formes sont désormais les mêmes. eux jusqu'à être retourné comme la perte finale

## Training SSD on a custom dataset

In [1]:
import os
if not os.path.exists('open-images-bus-trucks'):
    !pip install -q torch_snippets
    !wget --quiet https://www.dropbox.com/s/agmzwk95v96ihic/open-images-bus-trucks.tar.xz
    !tar -xf open-images-bus-trucks.tar.xz
    !rm open-images-bus-trucks.tar.xz
    !git clone https://github.com/sizhky/ssd-utils/
%cd ssd-utils

     |████████████████████████████████| 42 kB 1.4 MB/s 
     |████████████████████████████████| 56 kB 4.2 MB/s 
     |████████████████████████████████| 10.1 MB 27.0 MB/s 
     |████████████████████████████████| 57 kB 5.3 MB/s 
     |████████████████████████████████| 211 kB 44.9 MB/s 
     |████████████████████████████████| 51 kB 7.6 MB/s 
Cloning into 'ssd-utils'...
remote: Enumerating objects: 9, done.
remote: Counting objects: 100% (9/9), done.
remote: Compressing objects: 100% (8/8), done.
remote: Total 9 (delta 0), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (9/9), done.
/content/ssd-utils


In [3]:

from torch_snippets import *
DATA_ROOT = '../open-images-bus-trucks/'
IMAGE_ROOT = f'{DATA_ROOT}/images'
DF_RAW = df = pd.read_csv(f'{DATA_ROOT}/df.csv')

df = df[df['ImageID'].isin(df['ImageID'].unique().tolist())]

label2target = {l:t+1 for t,l in enumerate(DF_RAW['LabelName'].unique())}
label2target['background'] = 0
target2label = {t:l for l,t in label2target.items()}
background_class = label2target['background']
num_classes = len(label2target)

device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [4]:
import collections, os, torch
from PIL import Image
from torchvision import transforms


normalize = transforms.Normalize(
    mean=[0.485, 0.456, 0.406],
    std=[0.229, 0.224, 0.225]
)
denormalize = transforms.Normalize(
    mean=[-0.485/0.229, -0.456/0.224, -0.406/0.255],
    std=[1/0.229, 1/0.224, 1/0.255]
)

def preprocess_image(img):
    img = torch.tensor(img).permute(2,0,1)
    img = normalize(img)
    return img.to(device).float()
    
class OpenDataset(torch.utils.data.Dataset):
    w, h = 300, 300
    def __init__(self, df, image_dir=IMAGE_ROOT):
        self.image_dir = image_dir
        self.files = glob.glob(self.image_dir+'/*')
        self.df = df
        self.image_infos = df.ImageID.unique()
        logger.info(f'{len(self)} items loaded')
        
    def __getitem__(self, ix):
        # load images and masks
        image_id = self.image_infos[ix]
        img_path = find(image_id, self.files)
        img = Image.open(img_path).convert("RGB")
        img = np.array(img.resize((self.w, self.h), resample=Image.BILINEAR))/255.
        data = df[df['ImageID'] == image_id]
        labels = data['LabelName'].values.tolist()
        data = data[['XMin','YMin','XMax','YMax']].values
        data[:,[0,2]] *= self.w
        data[:,[1,3]] *= self.h
        boxes = data.astype(np.uint32).tolist() # convert to absolute coordinates
        return img, boxes, labels

    def collate_fn(self, batch):
        images, boxes, labels = [], [], []
        for item in batch:
            img, image_boxes, image_labels = item
            img = preprocess_image(img)[None]
            images.append(img)
            boxes.append(torch.tensor(image_boxes).float().to(device)/300.)
            labels.append(torch.tensor([label2target[c] for c in image_labels]).long().to(device))
        images = torch.cat(images).to(device)
        return images, boxes, labels
    def __len__(self):
        return len(self.image_infos)

In [6]:
import glob
from sklearn.model_selection import train_test_split
trn_ids, val_ids = train_test_split(df.ImageID.unique(), test_size=0.1, random_state=99)
trn_df, val_df = df[df['ImageID'].isin(trn_ids)], df[df['ImageID'].isin(val_ids)]
len(trn_df), len(val_df)

train_ds = OpenDataset(trn_df)
test_ds = OpenDataset(val_df)

train_loader = DataLoader(train_ds, batch_size=4, collate_fn=train_ds.collate_fn, drop_last=True)
test_loader = DataLoader(test_ds, batch_size=4, collate_fn=test_ds.collate_fn, drop_last=True)

2021-09-15 13:52:29.741 | INFO     | __main__:__init__:27 - 13702 items loaded
2021-09-15 13:52:29.789 | INFO     | __main__:__init__:27 - 1523 items loaded


In [7]:
def train_batch(inputs, model, criterion, optimizer):
    model.train()
    N = len(train_loader)
    images, boxes, labels = inputs
    _regr, _clss = model(images)
    loss = criterion(_regr, _clss, boxes, labels)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    return loss
    
@torch.no_grad()
def validate_batch(inputs, model, criterion):
    model.eval()
    images, boxes, labels = inputs
    _regr, _clss = model(images)
    loss = criterion(_regr, _clss, boxes, labels)
    return loss

In [8]:
from model import SSD300, MultiBoxLoss
from detect import *

In [9]:

n_epochs = 3

model = SSD300(num_classes, device)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4, weight_decay=1e-5)
criterion = MultiBoxLoss(priors_cxcy=model.priors_cxcy, device=device)

log = Report(n_epochs=n_epochs)
logs_to_print = 5

Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth


  0%|          | 0.00/528M [00:00<?, ?B/s]


Loaded base model.



/usr/local/lib/python3.7/dist-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='none' instead.
  warnings.warn(warning.format(ret))


In [ ]:
for epoch in range(n_epochs):
    _n = len(train_loader)
    for ix, inputs in enumerate(train_loader):
        loss = train_batch(inputs, model, criterion, optimizer)
        pos = (epoch + (ix+1)/_n)
        log.record(pos, trn_loss=loss.item(), end='\r')

    _n = len(test_loader)
    for ix,inputs in enumerate(test_loader):
        loss = validate_batch(inputs, model, criterion)
        pos = (epoch + (ix+1)/_n)
        log.record(pos, val_loss=loss.item(), end='\r')

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


EPOCH: 0.030	trn_loss: 4.434	(1262.23s - 124654.58s remaining)

In [ ]:
image_paths = Glob(f'{DATA_ROOT}/images/*')
image_id = choose(test_ds.image_infos)
img_path = find(image_id, test_ds.files)
original_image = Image.open(img_path, mode='r')
original_image = original_image.convert('RGB')

In [ ]:
image_paths = Glob(f'{DATA_ROOT}/images/*')
for _ in range(3):
    image_id = choose(test_ds.image_infos)
    img_path = find(image_id, test_ds.files)
    original_image = Image.open(img_path, mode='r')
    bbs, labels, scores = detect(original_image, model, min_score=0.9, max_overlap=0.5,top_k=200, device=device)
    labels = [target2label[c.item()] for c in labels]
    label_with_conf = [f'{l} @ {s:.2f}' for l,s in zip(labels,scores)]
    print(bbs, label_with_conf)
    show(original_image, bbs=bbs, texts=label_with_conf, text_sz=10)